# EVALUATION NOTEBOOK for TACAS 2025 PAPER

In [31]:
from load_data import load_data_mult

baseline_runconfig='NOTHING'

In [32]:
import helper_methods
import load_data
import pandas as pd
import matplotlib.pyplot as plt

import group_methods
import result_analysis
import numpy as np
import validate

# Load CSV

### Mult

In [33]:
df_raw= load_data.load_data_mult()

Found 2 CSV file(s) in the 'analysis_csv' directory.
Loaded: all_runs_new_SPLIT1_ws1_20241009_235902.csv
Number of specs in this file: 80
Loaded: all_runs_new_SPLIT2_ws2_20241009_221236.csv
Number of specs in this file: 98
Total number of files loaded: 2
Total Specs: 178
Final DataFrame shape: (1068, 88)


In [34]:
df = df_raw

In [35]:
df.head()

,Spec,RunConfig,UniqueIdentifier,TIMEOUT,ActionType,Result,CONFIG_TXT,INITIAL_BDD_TIME,HEURISTICS_TIME,SECOND_STEP_TIME,...,IMPORT,UNIQUE_PATTERNS,PATTERN_INSTANCES,TRIGGER_INSTANCES,MONITORS_INSTANCES,COUNTERS_INSTANCES,REDUCED_PATTERN,REDUCED_TRIGGER,SWAPPED_TO_ORIGINAL,ADDITIONAL
0,1670057023,NOTHING,1670057023.spectra,0,WELL_SEPARATION,NOT_STRONG,"TEMP: false, SORT_AFTERWARDS: false, GROUP_OPT...",1906,0,531,...,"import ""DwyerPatternsNew.spectra""",S_responds_to_P_globally,3,1,False,False,[],[],NaN,NaN
1,1670057023,FINAL,1670057023.spectra,0,WELL_SEPARATION,NOT_STRONG,"TEMP: false, SORT_AFTERWARDS: false, GROUP_OPT...",1859,94,375,...,"import ""DwyerPatternsNewReduced.spectra""",S_responds_to_P_globally,3,1,False,False,[],[REMOVED_HOLE],NaN,NaN
2,1670057023,NOTHING,1670057023.spectra,0,WELL_SEPARATION,NOT_STRONG,"TEMP: false, SORT_AFTERWARDS: false, GROUP_OPT...",1828,0,532,...,"import ""DwyerPatternsNew.spectra""",S_responds_to_P_globally,3,1,False,False,[],[],NaN,NaN
3,1670057023,FINAL,1670057023.spectra,0,WELL_SEPARATION,NOT_STRONG,"TEMP: false, SORT_AFTERWARDS: false, GROUP_OPT...",1860,109,375,...,"import ""DwyerPatternsNewReduced.spectra""",S_responds_to_P_globally,3,1,False,False,[],[REMOVED_HOLE],NaN,NaN
4,1670057023,NOTHING,1670057023.spectra,0,WELL_SEPARATION,NOT_STRONG,"TEMP: false, SORT_AFTERWARDS: false, GROUP_OPT...",1844,15,532,...,"import ""DwyerPatternsNew.spectra""",S_responds_to_P_globally,3,1,False,False,[],[],NaN,NaN


In [36]:
(df['Spec'] == '1670358342').any()

True

# Exploration #1
Before aggregation or handling the TIMEOUT

### Columns

In [37]:
df.columns

Index(['Spec', 'RunConfig', 'UniqueIdentifier', 'TIMEOUT', 'ActionType',
       'Result', 'CONFIG_TXT', 'INITIAL_BDD_TIME', 'HEURISTICS_TIME',
       'SECOND_STEP_TIME', 'PARALLEL_OVERHEAD_TIME', 'WORK_TIME', 'TOTAL_TIME',
       'PRE_ROYBDD_ORDER', 'POST_HEURISTICS_ORDER', 'POST_SECOND_STEP_ORDER',
       'POST_WORK_ORDER', 'PRE_ROYBDD_NODE_SIZE', 'POST_HEURISTICS_NODE_SIZE',
       'POST_SECOND_STEP_NODE_SIZE', 'POST_WORK_NODE_SIZE', 'AVERAGE_AUX_SIZE',
       'AVERAGE_SYS_DOM_IN_CONSTRAINT', 'AVERAGE_ENV_DOM_IN_CONSTRAINT',
       'AVERAGE_SYS_DOM_SIZE', 'AVERAGE_ENV_DOM_SIZE',
       'AVERAGE_CONSTRAINT_COMPLEXITY', 'INTERCONNECTNESS_INDEX',
       'TOTAL_REORDER_TIME', 'REORDER_CALL_AMOUNT', 'AVERAGE_REORDER_TIME',
       'AVERAGE_REORDER_GAIN', 'Z_FIXPOINTS', 'Y_FIXPOINTS', 'X_FIXPOINTS',
       'W_FIXPOINTS', 'REAL_CASES', 'UNREAL_CASES', 'REAL_TIME', 'UNREAL_TIME',
       'GRAPH_DATA', 'CLUSTER_INC_DATA', 'CLUSTER_VAR_DATA',
       'DOMAIN_GRAPH_DATA', 'CONSTRAINT_GRAPH_DATA', 

In [38]:
df[['WORK_TIME','ActionType']]

,WORK_TIME,ActionType
0,844,WELL_SEPARATION
1,1391,WELL_SEPARATION
2,890,WELL_SEPARATION
3,1406,WELL_SEPARATION
4,828,WELL_SEPARATION
...,...,...
1063,31,WELL_SEPARATION
1064,32,WELL_SEPARATION
1065,48,WELL_SEPARATION
1066,16,WELL_SEPARATION


In [39]:
print(len(df['Spec'].unique()))
df['Spec'].unique()

178


array(['1670057023', '1669965418', '1669902814', 'RobotsMPUnlabelled3',
       '1669665995', '1669987834', '1673819181', '1673821936',
       '1670596941', '1669983090', '1670075718', '1670169894',
       '1669915643', '1669983978', '1670075301', '1673547192',
       '1670168548', 'variant-2-1', '1673630794', '1669666055',
       'Junction-3', 'variant-3-4', '1673547609', '1669887914',
       '1673468315', 'variant-2-4', '1669468393', '1669813890',
       '1669640935', 'Cleaner-10', '1673469658', '1669982329',
       '1673738881', '1673548346', '1670062922', 'Cleaner-8', 'Cleaner-6',
       'variant-2-8', '1670597940', '1669981674', '1670597788',
       '1669966004', 'variant-3-9', '1669666438', '1670061890',
       '1670075033', '1670598571', '1669643605', 'Cleaner-3',
       '1673985257', 'variant-3-1', '1673467002', '1673686645',
       '1670066785', '1669968413', '1672860311', '1669902169',
       '1669966483', '1673821729', '1673824914', '1673687376',
       '1669901854', '1669966

In [40]:
df[['UNIQUE_PATTERNS','PATTERN_INSTANCES','TRIGGER_INSTANCES']]

,UNIQUE_PATTERNS,PATTERN_INSTANCES,TRIGGER_INSTANCES
0,S_responds_to_P_globally,3,1
1,S_responds_to_P_globally,3,1
2,S_responds_to_P_globally,3,1
3,S_responds_to_P_globally,3,1
4,S_responds_to_P_globally,3,1
...,...,...,...
1063,NaN,0,2
1064,NaN,0,2
1065,NaN,0,2
1066,NaN,0,2


### Possible Results

In [41]:
df['Result'].drop_duplicates()

0     NOT_STRONG
18        STRONG
Name: Result, dtype: object

### Possible RunConfigs

In [42]:
# All configs
unique_configs = df['RunConfig'].unique()
print("Unique RunConfigs:")
for config in unique_configs:
    print(f"- {config}")

Unique RunConfigs:
- NOTHING
- FINAL


### Has missing in non-TIMEOUT

In [43]:
if ((df[df['TIMEOUT']==0]['WORK_TIME'].isnull().values.any())):
  raise ValueError('missing value')

### Unique RunConfigs per Spec

In [44]:
# Count RunConfigs per Spec
config_counts = df.groupby('Spec')['RunConfig'].count()

print("Number of RunConfigs per Spec:")
print(config_counts)

print(f"\nUnique counts of RunConfigs per Spec: {config_counts.unique()}")

print("\nSummary statistics of RunConfigs per Spec:")
print(config_counts.describe())

Number of RunConfigs per Spec:
Spec
1669468393     6
1669640935     6
1669643093     6
1669643605     6
1669665995     6
              ..
variant-2-4    6
variant-2-8    6
variant-3-1    6
variant-3-4    6
variant-3-9    6
Name: RunConfig, Length: 178, dtype: int64

Unique counts of RunConfigs per Spec: [6]

Summary statistics of RunConfigs per Spec:
count    178.0
mean       6.0
std        0.0
min        6.0
25%        6.0
50%        6.0
75%        6.0
max        6.0
Name: RunConfig, dtype: float64


### Misc

In [45]:
df[['Spec','RunConfig','Result','POST_SECOND_STEP_NODE_SIZE','WORK_TIME']]

,Spec,RunConfig,Result,POST_SECOND_STEP_NODE_SIZE,WORK_TIME
0,1670057023,NOTHING,NOT_STRONG,15877,844
1,1670057023,FINAL,NOT_STRONG,13521,1391
2,1670057023,NOTHING,NOT_STRONG,15047,890
3,1670057023,FINAL,NOT_STRONG,13521,1406
4,1670057023,NOTHING,NOT_STRONG,15467,828
...,...,...,...,...,...
1063,1672060813,FINAL,STRONG,5787,31
1064,1672060813,NOTHING,STRONG,17238,32
1065,1672060813,FINAL,STRONG,6523,48
1066,1672060813,NOTHING,STRONG,18073,16


# Preprocess & Aggregation
We ran 3 times each Spec + RunConfig [except for TIMEOUT]. We need to aggregate it.

### Verify

In [46]:
# Group by Spec and RunConfig, compute Has_Timeout and Row_Count
grouped = df.groupby(['Spec', 'RunConfig']).agg(
    Has_Timeout=('TIMEOUT', lambda x: (x == 1).any()),
    Row_Count=('TIMEOUT', 'size')
).reset_index()

# Filter groups where Has_Timeout is True
timeout_counts = grouped[grouped['Has_Timeout']]

# Check if all counts are 1
all_ones = (timeout_counts['Row_Count'] == 1).all()

print(f"All Spec+RunConfig combinations with TIMEOUT=1 occur only once: {all_ones}")

All Spec+RunConfig combinations with TIMEOUT=1 occur only once: True


In [47]:
grouped


,Spec,RunConfig,Has_Timeout,Row_Count
0,1669468393,FINAL,False,3
1,1669468393,NOTHING,False,3
2,1669640935,FINAL,False,3
3,1669640935,NOTHING,False,3
4,1669643093,FINAL,False,3
...,...,...,...,...
351,variant-3-1,NOTHING,False,3
352,variant-3-4,FINAL,False,3
353,variant-3-4,NOTHING,False,3
354,variant-3-9,FINAL,False,3


# Validation

## Validation Same Across Spec + Action
Validation fields that should be equal for every Spec + Action rows

In [48]:
# Usage: RESULT is now ['Result']
RESULT = ['Result']

SAME_VARS_1 = ['TOTAL_VARS', 'SYS_VARS', 'ENV_VARS', 'AUX_VARS', 'TOTAL_DOMS', 'SYS_DOMS', 'ENV_DOMS', 'AUX_DOMS']
               
               
VAR_META_DATA=[
             'AVERAGE_AUX_SIZE', 'AVERAGE_SYS_DOM_IN_CONSTRAINT', 'AVERAGE_ENV_DOM_IN_CONSTRAINT',
             'AVERAGE_SYS_DOM_SIZE', 'AVERAGE_ENV_DOM_SIZE']

SAME_CONSTRAINTS = ['UNUSED_CONS_VARS']

BDD = ['BDDPACKAGE', 'FACTORY_TYPE']

FIXPOINT = ['Z_FIXPOINTS', 'Y_FIXPOINTS', 'X_FIXPOINTS', 'W_FIXPOINTS']

SAT_RESULT = ['Z_SAT_COUNT', 'Z_SAT_COUNT_START_S0']
SAT_GM = ['TRANS_ENV_SAT', 'TRANS_SYS_SAT',
       'JUSTS_SYS_SAT', 'JUSTS_ENV_SAT', 'INI_ENV_SAT', 'INI_SYS_SAT']

CASES = ['REAL_CASES', 'UNREAL_CASES']

GRAPH = ['GRAPH_DATA', 'CLUSTER_INC_DATA', 'CLUSTER_VAR_DATA', 'DOMAIN_GRAPH_DATA', 'CONSTRAINT_GRAPH_DATA']

GAMEMODEL = ['JUST_STRING_COUNT', 'JUST_STRING_ENV_COUNT', 'TRANS_STRING_SYS_COUNT',
             'TRANS_STRING_ENV_COUNT', 'INI_STRING_ENV_COUNT', 'INI_STRING_SYS_COUNT']

# Filter out timeout cases
df_no_timeout = helper_methods.get_df_specs_no_timeout(df)

In [49]:
df_no_timeout[df_no_timeout['Spec'] == '1670183448'][['RunConfig','Z_SAT_COUNT', 'Z_SAT_COUNT_START_S0']]

,RunConfig,Z_SAT_COUNT,Z_SAT_COUNT_START_S0
450,NOTHING,NaN,NaN
451,FINAL,NaN,NaN
452,NOTHING,NaN,NaN
453,FINAL,NaN,NaN
454,NOTHING,NaN,NaN
455,FINAL,NaN,NaN


In [50]:
df_no_timeout_temp1 = df_no_timeout[df_no_timeout['RunConfig'] == 'NOTHING']

In [51]:
# Results
validate.verify_columns_same_in_groups(df_no_timeout_temp1, RESULT,"RESULT")
validate.verify_columns_same_in_groups(df_no_timeout_temp1,SAT_RESULT,"SAT_RESULT")
validate.verify_columns_same_in_groups(df_no_timeout_temp1,SAT_GM,"SAT_GM")

All values in Result are the same for each Spec + Action combination.
All specified columns have consistent values within each Spec + Action combination.
All values in Z_SAT_COUNT are the same for each Spec + Action combination, with some all-NaN groups.
All values in Z_SAT_COUNT_START_S0 are the same for each Spec + Action combination, with some all-NaN groups.
All specified columns have consistent values within each Spec + Action combination.
All values in TRANS_ENV_SAT are the same for each Spec + Action combination.
All values in TRANS_SYS_SAT are the same for each Spec + Action combination.
All values in JUSTS_SYS_SAT are the same for each Spec + Action combination.
All values in JUSTS_ENV_SAT are the same for each Spec + Action combination.
All values in INI_ENV_SAT are the same for each Spec + Action combination.
All values in INI_SYS_SAT are the same for each Spec + Action combination.
All specified columns have consistent values within each Spec + Action combination.


True

In [52]:
validate.verify_columns_same_in_groups(df_no_timeout, CASES,"CASES")
validate.verify_columns_same_in_groups(df_no_timeout,GAMEMODEL,"GAMEMODEL")

All values in REAL_CASES are the same for each Spec + Action combination.
All values in UNREAL_CASES are the same for each Spec + Action combination.
All specified columns have consistent values within each Spec + Action combination.
All values in JUST_STRING_COUNT are the same for each Spec + Action combination, with some all-NaN groups.
All values in JUST_STRING_ENV_COUNT are the same for each Spec + Action combination, with some all-NaN groups.
All values in TRANS_STRING_SYS_COUNT are the same for each Spec + Action combination, with some all-NaN groups.
All values in TRANS_STRING_ENV_COUNT are the same for each Spec + Action combination, with some all-NaN groups.
All values in INI_STRING_ENV_COUNT are the same for each Spec + Action combination, with some all-NaN groups.
All values in INI_STRING_SYS_COUNT are the same for each Spec + Action combination, with some all-NaN groups.
All specified columns have consistent values within each Spec + Action combination.


True

In [53]:
# Model
validate.verify_columns_same_in_groups(df_no_timeout, SAME_CONSTRAINTS,"SAME_CONSTRAINTS")
validate.verify_columns_same_in_groups(df_no_timeout, FIXPOINT,"FIXPOINT")
#verify_columns_same_in_groups(df,GRAPH)

All values in UNUSED_CONS_VARS are the same for each Spec + Action combination, with some all-NaN groups.
All specified columns have consistent values within each Spec + Action combination.
All values in Z_FIXPOINTS are the same for each Spec + Action combination, with some all-NaN groups.
All values in Y_FIXPOINTS are the same for each Spec + Action combination, with some all-NaN groups.
All values in X_FIXPOINTS are the same for each Spec + Action combination, with some all-NaN groups.
All values in W_FIXPOINTS are the same for each Spec + Action combination, with some all-NaN groups.
All specified columns have consistent values within each Spec + Action combination.


True

In [54]:
# Misc
validate.verify_columns_same_in_groups(df_no_timeout, BDD,"BDD")
validate.verify_columns_same_in_groups(df_no_timeout,GRAPH,"GRAPH")


All values in BDDPACKAGE are the same for each Spec + Action combination.
All values in FACTORY_TYPE are the same for each Spec + Action combination.
All specified columns have consistent values within each Spec + Action combination.
All values in GRAPH_DATA are the same for each Spec + Action combination, with some all-NaN groups.
All values in CLUSTER_INC_DATA are the same for each Spec + Action combination, with some all-NaN groups.
All values in CLUSTER_VAR_DATA are the same for each Spec + Action combination, with some all-NaN groups.
All values in DOMAIN_GRAPH_DATA are the same for each Spec + Action combination, with some all-NaN groups.
All values in CONSTRAINT_GRAPH_DATA are the same for each Spec + Action combination, with some all-NaN groups.
All specified columns have consistent values within each Spec + Action combination.


True

In [55]:
df['CLUSTER_INC_DATA']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1063   NaN
1064   NaN
1065   NaN
1066   NaN
1067   NaN
Name: CLUSTER_INC_DATA, Length: 1068, dtype: float64

## Check VARS

In [56]:
SAME_DOMS = ['SYS_DOMS', 'ENV_DOMS']


In [57]:
def verify_permutation(df):
    df.groupby()
def str_to_ls(str):
    ls = str.split(',')
    list=  [item.replace('"', "") for item in ls]
    return list

df_no_timeout_temp = df_no_timeout.copy()

for column in SAME_DOMS:
    df_no_timeout_temp[column] = df_no_timeout_temp[column].apply(lambda x:str_to_ls(x))


In [58]:
df_no_timeout_temp['ENV_DOMS']

0       [targetAY, targetAY', targetCY, targetCY', tar...
1       [$PATTERN:2.ENV_CONSTRAINT.2.S_responds_to_P_g...
2       [$PATTERN:1.ENV_CONSTRAINT.1.S_responds_to_P_g...
3       [$PATTERN:2.ENV_CONSTRAINT.2.S_responds_to_P_g...
4       [$PATTERN:1.ENV_CONSTRAINT.1.S_responds_to_P_g...
                              ...                        
1063    [lambY0, lambY0', lambX1, lambX1', lambX0, lam...
1064    [lambX1, lambX1', lambY1, lambY1', lambX0, lam...
1065    [lambY0, lambY0', lambX1, lambX1', lambX0, lam...
1066    [lambX0, lambX0', elevatorY, elevatorY', lambY...
1067    [lambY0, lambY0', lambY1, lambY1', lambX0, lam...
Name: ENV_DOMS, Length: 1068, dtype: object

In [59]:
validate.verify_columns_permutations_in_groups(df_no_timeout_temp, SAME_DOMS,"SAME_DOMS")
#validate.verify_columns_permutations_in_groups(df_no_timeout, VAR_META_DATA,"VAR_META_DATA")


All lists in SYS_DOMS are permutations of each other for each Spec + ActionType combination.
All lists in ENV_DOMS are permutations of each other for each Spec + ActionType combination.
All specified columns have consistent lists (up to permutation) within each Spec + ActionType combination.


True

In [60]:
df['ENV_DOMS']

0       targetAY,targetAY',targetCY,targetCY',targetBX...
1       $PATTERN:2.ENV_CONSTRAINT.2.S_responds_to_P_gl...
2       $PATTERN:1.ENV_CONSTRAINT.1.S_responds_to_P_gl...
3       $PATTERN:2.ENV_CONSTRAINT.2.S_responds_to_P_gl...
4       $PATTERN:1.ENV_CONSTRAINT.1.S_responds_to_P_gl...
                              ...                        
1063    lambY0,lambY0',lambX1,lambX1',lambX0,lambX0',l...
1064    lambX1,lambX1',lambY1,lambY1',lambX0,lambX0',e...
1065    lambY0,lambY0',lambX1,lambX1',lambX0,lambX0',l...
1066    lambX0,lambX0',elevatorY,elevatorY',lambY1,lam...
1067    lambY0,lambY0',lambY1,lambY1',lambX0,lambX0',e...
Name: ENV_DOMS, Length: 1068, dtype: object